In [1]:
from Mollification import *

import trimesh
import os
import time

from buildGlueMap import *

root_folder = os.getcwd()

In [1]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            # root_folder, "../../GC/TestEdgeLengths", "11.stl"))
            # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93842.stl"))
            root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93743.stl"))


NameError: name 'igl' is not defined

In [3]:
delta = 1e-2
FL0 = igl.edge_lengths(V, F)

In [4]:
# time before
end0 = time.time()
FL1, nMoll1, _ = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end1 = time.time()
print("Constant Epsilon took: ", end1 - end0)

FL2, nMoll2, _ = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
end2 = time.time()
print("One by one step took: ", end2 - end1)

FL3, nMoll3, _ = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
end3 = time.time()
print("One by one interpolated took: ", end3 - end2)

FL4, nMoll4, _ = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
end4 = time.time()
print("Local least mollification Manhattan took: ", end4 - end3)

FL5, nMoll5, _ = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
end5 = time.time()
print("Local least mollification Euclidean took: ", end5 - end4)

Constant Epsilon took:  0.05582547187805176
One by one step took:  1.860107421875
One by one interpolated took:  1.7424280643463135
Local least mollification Manhattan took:  2.2255046367645264
Local least mollification Euclidean took:  4.129610538482666


In [5]:
end5 = time.time()

FL6, nMoll6, nIter6 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
end6 = time.time()
print("Sequential global, one by one step, mean pooling took: ", end6 - end5)

FL7, nMoll7, nIter7 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
end7 = time.time()
print("Sequential global, one by one interpolated, mean pooling took: ", end7 - end6)

FL8, nMoll8, nIter8 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
end8 = time.time()
print("Sequential global, local least mollification Manhattan, mean pooling took: ", end8 - end7)

FL9, nMoll9, nIter9 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
end9 = time.time()
print("Sequential global, local least mollification Euclidean, mean pooling took: ", end9 - end8)

0.009486183098360454
Iteration:  1 Mollifications:  621 Total Mollifications:  621
Iteration:  2 Mollifications:  0 Total Mollifications:  621
Sequential global, one by one step, mean pooling took:  13.83472204208374
0.009486183098360454
Iteration:  1 Mollifications:  621 Total Mollifications:  621
Iteration:  2 Mollifications:  0 Total Mollifications:  621
Sequential global, one by one interpolated, mean pooling took:  13.782729864120483
0.009486183098360454
Iteration:  1 Mollifications:  621 Total Mollifications:  621
Iteration:  2 Mollifications:  0 Total Mollifications:  621
Sequential global, local least mollification Manhattan, mean pooling took:  14.68224287033081
0.009486183098360454
Iteration:  1 Mollifications:  621 Total Mollifications:  621
Iteration:  2 Mollifications:  0 Total Mollifications:  621
Sequential global, local least mollification Euclidean, mean pooling took:  16.009372234344482


In [6]:
print("Constant Epsilon nMoll", nMoll1)
print("One by one step nMoll", nMoll2)
print("One by one interpolated nMoll", nMoll3)
print("Local least mollification Manhattan", nMoll4)
print("Local least mollification Euclidean", nMoll5) 
print("Sequential global, one by one step, mean pooling", nMoll6, "with", nIter6, "iterations")
print("Sequential global, one by one interpolated, mean pooling", nMoll7, "with", nIter7, "iterations")
print("Sequential global, local least mollification Manhattan, mean pooling", nMoll8, "with", nIter8, "iterations")
print("Sequential global, local least mollification Euclidean, mean pooling", nMoll9, "with", nIter9, "iterations")

Constant Epsilon nMoll 699466
One by one step nMoll 621
One by one interpolated nMoll 621
Local least mollification Manhattan 621
Local least mollification Euclidean 621
Sequential global, one by one step, mean pooling 621 with 2 iterations
Sequential global, one by one interpolated, mean pooling 621 with 2 iterations
Sequential global, local least mollification Manhattan, mean pooling 621 with 2 iterations
Sequential global, local least mollification Euclidean, mean pooling 621 with 2 iterations


In [7]:
print(np.sum(FL1 - FL0) / np.sum(FL0))
print(np.sum(FL2 - FL0) / np.sum(FL0))
print(np.sum(FL3 - FL0) / np.sum(FL0))
print(np.sum(FL4 - FL0) / np.sum(FL0))
print(np.sum(FL5 - FL0) / np.sum(FL0))
print(np.sum(FL6 - FL0) / np.sum(FL0))
print(np.sum(FL7 - FL0) / np.sum(FL0))
print(np.sum(FL8 - FL0) / np.sum(FL0))
print(np.sum(FL9 - FL0) / np.sum(FL0))

0.009999969844763803
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06


In [9]:
F = np.array([F]).repeat(2, axis=0)
print(F.shape)

(2, 144046, 3)


In [8]:
b = np.array([- delta,   - delta,    - delta,    0,     0,    0])
print(b, b[3])

[-0.1 -0.1 -0.1  0.   0.   0. ] 0.0


In [2]:
# read paths from a text file, each line is a path
paths = []
# make sure encoding = "utf-16 le" for the tooNiceMeshPaths.txt file


# with open("filtering-thing10k/veryBadMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

with open("filtering-thing10k/validMeshPaths.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())
with open("filtering-thing10k/tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())
with open("filtering-thing10k/large_tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())
with open("filtering-thing10k/large_kindaBadMeshPaths.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())

# randomly order paths
import random
random.shuffle(paths)


print("Number of paths: ", len(paths))


Number of paths:  913


In [5]:

print(times)
print(times[0][0])

[[[0.008999824523925781, 0.009971857070922852, 0.0074045658111572266, 0.007978677749633789, 0.007978200912475586, 0.007978677749633789, 0.006981372833251953, 0.0069806575775146484, 0.007978200912475586, 0.00698399543762207, 0.0020787715911865234, 0.01875782012939453, 0.012966156005859375, 0.20042777061462402, 0.21538996696472168, 0.22843718528747559, 0.24833154678344727, 0.22535395622253418, 0.23342514038085938, 0.2054445743560791, 0.22211956977844238, 0.22598624229431152, 0.21291351318359375, 0.22838973999023438, 0.2183852195739746, 0.2104785442352295, 0.0209505558013916, 0.0209810733795166, 0.02093958854675293, 0.02194070816040039, 0.02194833755493164, 0.020937442779541016, 0.02293872833251953, 0.024903535842895508, 0.02397322654724121, 0.023938894271850586, 0.020876646041870117, 0.025930166244506836, 0.025974512100219727, 0.042885780334472656, 0.04487967491149902, 0.044912099838256836, 0.04484891891479492, 0.05186605453491211, 0.04487323760986328, 0.040891408920288086, 0.04391884803

In [8]:
delta = [1e-5, 1e-4, 1e-3, 1e-2] #, 1e-3, 1e-2, 1e-1]

def func(FL, G, delta, i):
    if i == 0:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
    elif i == 1:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
    elif i == 2:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
    elif i == 3:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
    elif i == 4:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
    elif i == 5:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 6:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 7:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 8:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 9:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 10:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 11:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 12:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MAX)


time_glue = []

moll_labels = [
    "Constant Epsilon",
    "One by One Step (Local)",
    "One by One Interpolated (Local)",
    "Local Least Mollification Manhattan (Local)",
    "Local Least Mollification Euclidean (Local)"
    # "One by One Step (Global, Mean)",
    # "One by One Interpolated (Global, Mean)",
    # "Local Least Mollification Manhattan (Global, Mean)",
    # "Local Least Mollification Euclidean (Global, Mean)",
    # "One by One Step (Global, Max)",
    # "One by One Interpolated (Global, Max)",
    # "Local Least Mollification Manhattan (Global, Max)",
    # "Local Least Mollification Euclidean (Global, Max)"
]

times = np.zeros((len(moll_labels), len(delta)))

n_moll = np.zeros((len(moll_labels), len(delta)))

n_iter = np.zeros((len(moll_labels), len(delta)))

avg_moll = np.zeros((len(moll_labels), len(delta)))

avg_moll_for_mollified_edges = np.zeros((13, len(delta)))



for i in range(len(paths)):

    [V,F]= igl.read_triangle_mesh(os.path.join(
                # root_folder, "../../GC/TestEdgeLengths", "11.stl"))
                # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93842.stl"))
                root_folder, "../../GC/", paths[i]))

    
    start = time.time()
    G = build_gluing_map(F)
    end = time.time()
    time_glue.append(end - start)
    
    FL0 = igl.edge_lengths(V, F)

    avg_edge_length = np.mean(FL0)

    for j in range(len(delta)):
        for k in range(len(moll_labels)):
            FL = np.copy(FL0)
            start = time.time()
            FL, nMoll, nIter = func(FL, G, delta[j], k)
            end = time.time()
            times[k][j] += end - start
            n_moll[k][j] += nMoll / len(FL0 * 3)
            n_iter[k][j] += nIter
            avg_moll[k][j] += np.sum(FL - FL0) / np.sum(FL0)
            if(nMoll > 0):
                avg_moll_for_mollified_edges[k][j] += np.sum(FL - FL0) / (avg_edge_length * nMoll * 3)
            print(i, j, k)


    if(i % 10 == 0 and i != 0):
        print("Finished ", i + 1, " meshes.")
        print("Average time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
        print("_________________________________________________________")

        for j in range(len(delta)):
            for k in range(len(moll_labels)):
                print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
                print("Average time: ", times[k][j] / (i + 1))
                print("Average number of mollified edges: ", n_moll[k][j] / (i + 1))
                print("Average number of iterations: ", n_iter[k][j] / (i + 1))
                print("Average mollification: ", avg_moll[k][j] / (i + 1))
                print("Average mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / (i + 1))
                print("_________________________________________________________")
            
        

print("Finished All meshes.")
print("Average time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
print("_________________________________________________________")
for j in range(len(delta)):
    for k in range(len(moll_labels)):
        print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
        print("Average time: ", times[k][j] / len(paths))
        print("Average number of mollified edges: ", n_moll[k][j] / len(paths))
        print("Average number of iterations: ", n_iter[k][j] / len(paths))
        print("Average mollification: ", avg_moll[k][j] / len(paths))
        print("Average mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / len(paths))
        print("_________________________________________________________")

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 3 0
2 3 1
2 3 2
2 3 3
